# Time Series Analysis of NYC Yellow Taxi Trips

In [ ]:
%pip install kagglehub
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install statsmodels
%pip install pmdarima

import kagglehub
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from pmdarima import auto_arima  # Per ottimizzazione automatica dei parametri

## Dataset loading

In [ ]:
path = kagglehub.dataset_download("elemento/nyc-yellow-taxi-trip-data")
file_path1 = os.path.join(path, "yellow_tripdata_2015-01.csv")
file_path2 = os.path.join(path, "yellow_tripdata_2016-01.csv")
file_path3 = os.path.join(path, "yellow_tripdata_2016-02.csv")
file_path4 = os.path.join(path, "yellow_tripdata_2016-03.csv")


In [ ]:
# Columns of interest
columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'fare_amount', 'total_amount']

# Load the dataset into a pandas dataframe
dfs = [pd.read_csv(f, usecols=columns, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']) for f in 
       [file_path1, file_path2, file_path3, file_path4]]
df = pd.concat(dfs, ignore_index=True)

## Data cleaning

In [ ]:
df.dropna(subset=columns, inplace=True)
df = df[(df['trip_distance'] > 0) & (df['fare_amount'] > 0) & (df['total_amount'] > 0)]

## Time variables creation

In [ ]:
df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df = df[(df['trip_duration'] > 0) & (df['trip_duration'] < 240)]

In [ ]:
# Calculate the average duration of trips for each day
daily_avg_duration = df.groupby('pickup_date')['trip_duration'].mean()

# Visualize the trend of the average trip duration
plt.figure(figsize=(14, 7))
daily_avg_duration.plot()
plt.title('Average trip duration per day')
plt.xlabel('Date')
plt.ylabel('Average trip duration (minutes)')
plt.show()

## Stationarity analysis

In [ ]:
daily_trips = df.groupby('pickup_date').size()

ts_daily_trips = daily_trips.copy()
ts_daily_trips.index = pd.to_datetime(ts_daily_trips.index)
ts_daily_trips = ts_daily_trips.asfreq('D', fill_value=0)

In [ ]:
plt.figure(figsize=(14, 7))
ts_daily_trips.plot()
plt.title('Daily NYC Taxi Trips')
plt.xlabel('Date')
plt.ylabel('Number of Trips')
plt.grid()
plt.show()

In [ ]:
def test_stationarity_with_interpretation(timeseries):
    result = adfuller(timeseries)
    print('=== Augmented Dickey-Fuller Test ===')
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.4f}')
    
    # Interpretazione del risultato
    if result[1] <= 0.05:
        print("\n✅ La serie è stazionaria")
    else:
        print("\n❌ La serie NON è stazionaria")

# Test sulla serie originale
print("\n[TEST SULLA SERIE ORIGINALE]")
test_stationarity_with_interpretation(ts_daily_trips)

In [ ]:
ts_daily_trips_diff = ts_daily_trips.diff().dropna()
print("\n[TEST SULLA SERIE DIFFERENZIATA]")
test_stationarity_with_interpretation(ts_daily_trips_diff)

The $p$-value is less than $0.05$, which means that we can reject the null hypothesis that the time series is non-stationary. The time series is stationary.

In [ ]:
plt.figure(figsize=(14, 7))
plt.subplot(2, 1, 1)
ts_daily_trips.plot(ax=plt.gca())
plt.title('Serie Originale')
plt.xlabel('Date')
plt.ylabel('Numero di Viaggi')
plt.grid()

plt.subplot(2, 1, 2)
ts_daily_trips_diff.plot(ax=plt.gca(), color='orange')
plt.title('Serie Differenziata (Primo Ordine)')
plt.xlabel('Date')
plt.ylabel('Differenza Numero di Viaggi')
plt.grid()
plt.tight_layout()
plt.show()

### Automatic optimization of parameters with auto_arima

In [ ]:
auto_arima_model = auto_arima(ts_daily_trips, seasonal=True, m=7, trace=True, error_action='ignore', suppress_warnings=True)
print("Best ARIMA model:", auto_arima_model.summary())

## ARIMA-SARIMA models
### p, d, q parameters

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(ts_daily_trips_diff, ax=plt.gca(), lags=20)
plt.subplot(122)
plot_pacf(ts_daily_trips_diff, ax=plt.gca(), lags=20)
plt.show()

In [ ]:
model = ARIMA(ts_daily_trips_diff, order=(4,1,2))

arima_fit = model.fit()

print("\n=== ARIMA Model Summary ===")
print(arima_fit.summary())

In [ ]:
# Diagnostic plots
arima_fit.plot_diagnostics(figsize=(12,8))
plt.show()

In [ ]:
model_sarima = sm.tsa.statespace.SARIMAX(ts_daily_trips, order=(4, 1, 2))
sarima_fit = model_sarima.fit()

print(sarima_fit.summary())

In [ ]:
sarima_fit.plot_diagnostics(figsize=(12, 8))
plt.show()

In [ ]:
forecast_arima = arima_fit.get_forecast(steps=7)
forecast_sarima = sarima_fit.get_forecast(steps=7)
forecast_arima_ci = forecast_arima.conf_int()
forecast_sarima_ci = forecast_sarima.conf_int()

plt.figure(figsize=(14, 7))
plt.plot(ts_daily_trips, label='Observed')
plt.plot(forecast_arima.predicted_mean.index, forecast_arima.predicted_mean.values, label='ARIMA Forecast', color='red')
plt.fill_between(forecast_arima_ci.index, forecast_arima_ci.iloc[:, 0], forecast_arima_ci.iloc[:, 1], color='red', alpha=0.3)
plt.plot(forecast_sarima.predicted_mean.index, forecast_sarima.predicted_mean.values, label='SARIMA Forecast', color='green')
plt.fill_between(forecast_sarima_ci.index, forecast_sarima_ci.iloc[:, 0], forecast_sarima_ci.iloc[:, 1], color='green', alpha=0.3)
plt.title('Forecast Comparison')
plt.xlabel('Date')
plt.ylabel('Number of Trips')
plt.legend()
plt.grid()
plt.show()

- La previsione ARIMA rappresenta i valori differenziati. Per tornare alla scala originale, dovremmo applicare l'inverso della differenziazione.
- Gli intervalli di confidenza visualizzati indicano l'incertezza associata alla previsione.
- Rispetto ad ARIMA, SARIMA considera anche la stagionalità (settimanale in questo caso).

## Hour based analysis

Escludiamo il file CSV relativo a gennaio 2015 in modo tale da ottenere una sottosezione dell'intero dataset relativa al periodo gennaio2016-febbario2016. In questo modo, in quell'intervallo i dati potranno essere aggragati su base oraria con intervalli uniformi tra ogni osservazione

In [ ]:
# Specificare i file rilevanti per gennaio, febbraio e marzo 2016
hour_dfs = [pd.read_csv(f, usecols=columns, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']) 
            for f in [file_path2, file_path3, file_path4]]
hour_df = pd.concat(hour_dfs, ignore_index=True)

In [ ]:
# Rimuovere righe con valori nulli
hour_df.dropna(subset=columns, inplace=True)

# Filtrare viaggi con valori negativi o anomali
hour_df = hour_df[(hour_df['trip_distance'] > 0) & 
                  (hour_df['fare_amount'] > 0) & 
                  (hour_df['total_amount'] > 0)]

# Calcolare la durata del viaggio
hour_df['trip_duration'] = (hour_df['tpep_dropoff_datetime'] - hour_df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filtrare viaggi con durata negativa o eccessiva (>4 ore)
hour_df = hour_df[(hour_df['trip_duration'] > 0) & (hour_df['trip_duration'] < 240)]

In [ ]:
# Creare un indice temporale arrotondando al più vicino inizio dell'ora
hour_df['pickup_hour'] = hour_df['tpep_pickup_datetime'].dt.floor('H')

# Aggregare il numero di viaggi per ogni ora
hourly_trips = hour_df.groupby('pickup_hour').size()

# Trasformare in una serie temporale con frequenza fissa
hourly_ts = hourly_trips.asfreq('H', fill_value=0)

In [ ]:
plt.figure(figsize=(14, 7))
hourly_ts.plot()
plt.title('Numero di viaggi per ora (gennaio-febbraio 2016)')
plt.xlabel('Ora')
plt.ylabel('Numero di viaggi')
plt.grid()
plt.show()


In [ ]:
print("\n[TEST SULLA SERIE ORIGINALE]")
test_stationarity_with_interpretation(hourly_ts)

In [ ]:
auto_arima_model = auto_arima(hourly_ts, seasonal=True, m=7, trace=True, error_action='ignore', suppress_warnings=True)
print("Best ARIMA model:", auto_arima_model.summary())

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(hourly_ts, ax=plt.gca(), lags=20)
plt.subplot(122)
plot_pacf(hourly_ts, ax=plt.gca(), lags=20)
plt.show()

In [ ]:
hourly_arima_model = ARIMA(hourly_ts, order=(4,1,2), seasonal_order=(0, 0, 1, 24))

hourly_arima_fit = hourly_arima_model.fit()

print("\n=== ARIMA Model Summary ===")
print(hourly_arima_fit.summary())

In [ ]:
hourly_arima_fit.plot_diagnostics(figsize=(12,8))
plt.show()

In [ ]:
hourly_model_sarima = sm.tsa.statespace.SARIMAX(hourly_ts, order=(4, 1, 2), seasonal_order=(0, 0, 1, 7))
hourly_sarima_fit = hourly_model_sarima.fit()

print(hourly_sarima_fit.summary())

In [ ]:
hourly_sarima_fit.plot_diagnostics(figsize=(12, 8))
plt.show()

In [ ]:
hourly_forecast_arima = hourly_arima_fit.get_forecast(steps=24*3)
hourly_forecast_sarima = hourly_sarima_fit.get_forecast(steps=24*3)
hourly_forecast_arima_ci = hourly_forecast_arima.conf_int()
hourly_forecast_sarima_ci = hourly_forecast_sarima.conf_int()

plt.figure(figsize=(14, 7))
plt.plot(hourly_ts, label='Observed')
plt.plot(hourly_forecast_arima.predicted_mean.index, hourly_forecast_arima.predicted_mean.values, label='ARIMA Forecast', color='red')
plt.fill_between(hourly_forecast_arima_ci.index, hourly_forecast_arima_ci.iloc[:, 0], hourly_forecast_arima_ci.iloc[:, 1], color='red', alpha=0.3)
plt.plot(hourly_forecast_sarima.predicted_mean.index, hourly_forecast_sarima.predicted_mean.values, label='SARIMA Forecast', color='green')
plt.fill_between(hourly_forecast_sarima_ci.index, hourly_forecast_sarima_ci.iloc[:, 0], hourly_forecast_sarima_ci.iloc[:, 1], color='green', alpha=0.3)
plt.title('Forecast Comparison')
plt.xlabel('Date')
plt.ylabel('Number of Trips')
plt.legend()
plt.grid()
plt.show()